In [9]:
# -*- coding: utf-8 -*-

import urllib.request
import json
import re
import xlwt
import pandas as pd
import time


#设置代理IP
proxy_addr="120.83.123.165"

#定义匹配日期函数
def regexper_date(date):
    pattern = re.compile('2019-(0[1-9]|1[0-2])-(2[0-7]|1[0-9]|0[1-9])|201([0-8])-(0[1-9]|1[0-2])-(3[0-1]|2[0-9]|1[0-9]|0[1-9])')
    matchObj = not(re.match(pattern, date))
    return matchObj

#定义页面打开函数
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data

#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
    data=use_proxy(url,proxy_addr)
    content=json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if(data.get('tab_type')=='weibo'):
            containerid=data.get('containerid')
    return containerid

#获取微博大V账号的用户基本信息;
def get_userInfo(id):
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    data=use_proxy(url,proxy_addr)
    content=json.loads(data).get('data')
    name=content.get('userInfo').get('screen_name')
    print("微博昵称："+name+"\n")
    return name


#获取微博内容信息,并保存到文本中，内容包括：每条微博的内容、微博详情页面地址、点赞数、评论数、转发数等
def get_weibo(id,file,name,count):
    i=1
    key=True
    while True:
        url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
        weibo_url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id+'&containerid='+get_containerid(url)+'&page='+str(i)
        try:
            data=use_proxy(weibo_url,proxy_addr)
            content=json.loads(data).get('data')
            cards=content.get('cards')
            if(len(cards)>0)and(key==True):
                for j in range(len(cards)):
                    card_type=cards[j].get('card_type')
                    if(card_type==9):
                        mblog=cards[j].get('mblog')
                        attitudes_count=mblog.get('attitudes_count')
                        comments_count=mblog.get('comments_count')
                        created_at=mblog.get('created_at')
                        reposts_count=mblog.get('reposts_count')
                        scheme=cards[j].get('scheme')
                        text=mblog.get('text')
                        if(regexper_date(created_at)):
                            count+=1
                            print("-----正在爬取第"+str(i)+"页，第"+str(count)+"条微博------")
                            file.write(count,0,str(name))
                            file.write(count,1,str(created_at))
                            file.write(count,2,str(text))
                            file.write(count,3,str(attitudes_count))
                            file.write(count,4,str(comments_count))
                            file.write(count,5,str(reposts_count))
                            file.write(count,6,str(scheme))
                        else:
                            key=False
                            print('--------------爬取完毕--------------')
                            break
                i+=1
            else:
                return count
                break
        except Exception as e:
            print(e)
            pass

if __name__=="__main__":
    workbook = xlwt.Workbook(encoding = 'utf-8')
    worksheet = workbook.add_sheet('SinaData')
    style = xlwt.XFStyle()
    worksheet.write(0, 0, '微博名') 
    worksheet.write(0, 1, '发布时间') 
    worksheet.write(0, 2, '微博内容') 
    worksheet.write(0, 3, '点赞数') 
    worksheet.write(0, 4, '评论数') 
    worksheet.write(0, 5, '转发数')
    worksheet.write(0, 6, '微博地址')
    print("--------------创建SinaData成功--------------")
    df_uid = pd.read_excel('D:/name_uid.xlsx')
    uid = df_uid['uid'].values.tolist()
    print("--------------读取name_uid成功--------------")
    
    count = 0
    for z in uid: 
        #将浮点型转为int，再将int转换为string
        userID = str(int(z))
        try:
            name = get_userInfo(userID)
            count = get_weibo(userID, worksheet, name, count)
        except Exception as e:
            print(e)
            continue
        time.sleep(2)
    
    print('--------------抓取完毕--------------')
    print('------共抓取微博' + str(count) + "条------")
    workbook.save('D:\SinaData.xls')
    print('--------------保存成功--------------')
    

--------------创建SinaData成功--------------
--------------读取name_uid成功--------------
微博昵称：长沙市疾控中心

-----正在爬取第1页，第1条微博------
-----正在爬取第1页，第2条微博------
-----正在爬取第1页，第3条微博------
-----正在爬取第1页，第4条微博------
-----正在爬取第1页，第5条微博------
-----正在爬取第1页，第6条微博------
-----正在爬取第1页，第7条微博------
-----正在爬取第1页，第8条微博------
-----正在爬取第1页，第9条微博------
-----正在爬取第1页，第10条微博------
-----正在爬取第2页，第11条微博------
-----正在爬取第2页，第12条微博------
-----正在爬取第2页，第13条微博------
-----正在爬取第2页，第14条微博------
-----正在爬取第2页，第15条微博------
-----正在爬取第2页，第16条微博------
-----正在爬取第2页，第17条微博------
-----正在爬取第2页，第18条微博------
-----正在爬取第2页，第19条微博------
-----正在爬取第2页，第20条微博------
-----正在爬取第3页，第21条微博------
-----正在爬取第3页，第22条微博------
-----正在爬取第3页，第23条微博------
-----正在爬取第3页，第24条微博------
-----正在爬取第3页，第25条微博------
-----正在爬取第3页，第26条微博------
-----正在爬取第3页，第27条微博------
-----正在爬取第3页，第28条微博------
-----正在爬取第3页，第29条微博------
-----正在爬取第3页，第30条微博------
-----正在爬取第4页，第31条微博------
-----正在爬取第4页，第32条微博------
-----正在爬取第4页，第33条微博------
-----正在爬取第4页，第34条微博------
-----正在爬取第4页，第35条微博------
---

-----正在爬取第20页，第308条微博------
-----正在爬取第20页，第309条微博------
-----正在爬取第20页，第310条微博------
-----正在爬取第20页，第311条微博------
-----正在爬取第20页，第312条微博------
-----正在爬取第20页，第313条微博------
--------------爬取完毕--------------
微博昵称：中国结核病防治

--------------爬取完毕--------------
微博昵称：江苏疾控

-----正在爬取第1页，第314条微博------
-----正在爬取第1页，第315条微博------
-----正在爬取第1页，第316条微博------
-----正在爬取第1页，第317条微博------
-----正在爬取第1页，第318条微博------
-----正在爬取第1页，第319条微博------
-----正在爬取第1页，第320条微博------
-----正在爬取第1页，第321条微博------
-----正在爬取第1页，第322条微博------
-----正在爬取第1页，第323条微博------
-----正在爬取第2页，第324条微博------
-----正在爬取第2页，第325条微博------
-----正在爬取第2页，第326条微博------
-----正在爬取第2页，第327条微博------
-----正在爬取第2页，第328条微博------
-----正在爬取第2页，第329条微博------
-----正在爬取第2页，第330条微博------
-----正在爬取第2页，第331条微博------
-----正在爬取第2页，第332条微博------
-----正在爬取第2页，第333条微博------
-----正在爬取第3页，第334条微博------
-----正在爬取第3页，第335条微博------
-----正在爬取第3页，第336条微博------
-----正在爬取第3页，第337条微博------
-----正在爬取第3页，第338条微博------
-----正在爬取第3页，第339条微博------
-----正在爬取第3页，第340条微博------
-----正在爬取第3页

微博昵称：银川疾控中心

-----正在爬取第1页，第595条微博------
-----正在爬取第1页，第596条微博------
-----正在爬取第1页，第597条微博------
-----正在爬取第1页，第598条微博------
-----正在爬取第1页，第599条微博------
-----正在爬取第1页，第600条微博------
-----正在爬取第1页，第601条微博------
-----正在爬取第1页，第602条微博------
-----正在爬取第2页，第603条微博------
-----正在爬取第2页，第604条微博------
-----正在爬取第2页，第605条微博------
-----正在爬取第2页，第606条微博------
-----正在爬取第2页，第607条微博------
-----正在爬取第2页，第608条微博------
-----正在爬取第2页，第609条微博------
-----正在爬取第2页，第610条微博------
-----正在爬取第3页，第611条微博------
-----正在爬取第3页，第612条微博------
-----正在爬取第3页，第613条微博------
-----正在爬取第3页，第614条微博------
-----正在爬取第3页，第615条微博------
-----正在爬取第3页，第616条微博------
-----正在爬取第3页，第617条微博------
-----正在爬取第3页，第618条微博------
-----正在爬取第3页，第619条微博------
-----正在爬取第3页，第620条微博------
-----正在爬取第4页，第621条微博------
-----正在爬取第4页，第622条微博------
-----正在爬取第4页，第623条微博------
-----正在爬取第4页，第624条微博------
-----正在爬取第4页，第625条微博------
-----正在爬取第4页，第626条微博------
-----正在爬取第4页，第627条微博------
-----正在爬取第4页，第628条微博------
-----正在爬取第4页，第629条微博------
-----正在爬取第4页，第630条微博------
-----正在爬取第5页，第6

-----正在爬取第5页，第871条微博------
-----正在爬取第5页，第872条微博------
-----正在爬取第5页，第873条微博------
-----正在爬取第5页，第874条微博------
-----正在爬取第5页，第875条微博------
-----正在爬取第5页，第876条微博------
-----正在爬取第5页，第877条微博------
-----正在爬取第5页，第878条微博------
-----正在爬取第5页，第879条微博------
-----正在爬取第5页，第880条微博------
-----正在爬取第6页，第881条微博------
-----正在爬取第6页，第882条微博------
-----正在爬取第6页，第883条微博------
-----正在爬取第6页，第884条微博------
-----正在爬取第6页，第885条微博------
-----正在爬取第6页，第886条微博------
-----正在爬取第6页，第887条微博------
-----正在爬取第6页，第888条微博------
-----正在爬取第6页，第889条微博------
-----正在爬取第7页，第890条微博------
-----正在爬取第7页，第891条微博------
-----正在爬取第7页，第892条微博------
-----正在爬取第7页，第893条微博------
-----正在爬取第7页，第894条微博------
-----正在爬取第7页，第895条微博------
-----正在爬取第7页，第896条微博------
-----正在爬取第7页，第897条微博------
-----正在爬取第7页，第898条微博------
-----正在爬取第7页，第899条微博------
-----正在爬取第8页，第900条微博------
-----正在爬取第8页，第901条微博------
-----正在爬取第8页，第902条微博------
-----正在爬取第8页，第903条微博------
-----正在爬取第8页，第904条微博------
-----正在爬取第8页，第905条微博------
-----正在爬取第8页，第906条微博------
-----正在爬取第8页，第907条微博------
-

-----正在爬取第4页，第1139条微博------
-----正在爬取第4页，第1140条微博------
-----正在爬取第4页，第1141条微博------
-----正在爬取第4页，第1142条微博------
-----正在爬取第4页，第1143条微博------
-----正在爬取第4页，第1144条微博------
-----正在爬取第4页，第1145条微博------
-----正在爬取第4页，第1146条微博------
-----正在爬取第4页，第1147条微博------
-----正在爬取第4页，第1148条微博------
-----正在爬取第5页，第1149条微博------
-----正在爬取第5页，第1150条微博------
-----正在爬取第5页，第1151条微博------
-----正在爬取第5页，第1152条微博------
-----正在爬取第5页，第1153条微博------
-----正在爬取第5页，第1154条微博------
-----正在爬取第5页，第1155条微博------
-----正在爬取第5页，第1156条微博------
-----正在爬取第5页，第1157条微博------
-----正在爬取第6页，第1158条微博------
-----正在爬取第6页，第1159条微博------
-----正在爬取第6页，第1160条微博------
-----正在爬取第6页，第1161条微博------
-----正在爬取第6页，第1162条微博------
-----正在爬取第6页，第1163条微博------
-----正在爬取第6页，第1164条微博------
-----正在爬取第6页，第1165条微博------
-----正在爬取第6页，第1166条微博------
-----正在爬取第6页，第1167条微博------
-----正在爬取第7页，第1168条微博------
-----正在爬取第7页，第1169条微博------
-----正在爬取第7页，第1170条微博------
-----正在爬取第7页，第1171条微博------
-----正在爬取第7页，第1172条微博------
-----正在爬取第7页，第1173条微博------
-----正在爬取第7页，第1174条微

...

微博昵称：大兴创卫

微博昵称：健康莫力达瓦

--------------爬取完毕--------------
微博昵称：中国健康科普主场

-----正在爬取第2页，第27443条微博------
-----正在爬取第2页，第27444条微博------
-----正在爬取第2页，第27445条微博------
-----正在爬取第2页，第27446条微博------
-----正在爬取第2页，第27447条微博------
-----正在爬取第2页，第27448条微博------
-----正在爬取第2页，第27449条微博------
-----正在爬取第2页，第27450条微博------
-----正在爬取第2页，第27451条微博------
-----正在爬取第2页，第27452条微博------
-----正在爬取第3页，第27453条微博------
-----正在爬取第3页，第27454条微博------
-----正在爬取第3页，第27455条微博------
-----正在爬取第3页，第27456条微博------
-----正在爬取第3页，第27457条微博------
-----正在爬取第3页，第27458条微博------
-----正在爬取第3页，第27459条微博------
-----正在爬取第3页，第27460条微博------
-----正在爬取第3页，第27461条微博------
-----正在爬取第4页，第27462条微博------
-----正在爬取第4页，第27463条微博------
-----正在爬取第4页，第27464条微博------
-----正在爬取第4页，第27465条微博------
-----正在爬取第4页，第27466条微博------
-----正在爬取第4页，第27467条微博------
-----正在爬取第4页，第27468条微博------
-----正在爬取第4页，第27469条微博------
-----正在爬取第4页，第27470条微博------
-----正在爬取第4页，第27471条微博------
-----正在爬取第5页，第27472条微博------
-----正在爬取第5页，第27473条微博------
-----正在爬取第5页，第27474条微博------


-----正在爬取第20页，第27716条微博------
-----正在爬取第20页，第27717条微博------
-----正在爬取第20页，第27718条微博------
-----正在爬取第20页，第27719条微博------
-----正在爬取第20页，第27720条微博------
-----正在爬取第20页，第27721条微博------
-----正在爬取第20页，第27722条微博------
-----正在爬取第20页，第27723条微博------
-----正在爬取第20页，第27724条微博------
-----正在爬取第20页，第27725条微博------
-----正在爬取第21页，第27726条微博------
-----正在爬取第21页，第27727条微博------
-----正在爬取第21页，第27728条微博------
-----正在爬取第21页，第27729条微博------
-----正在爬取第21页，第27730条微博------
--------------爬取完毕--------------
微博昵称：北京市社区卫生服务管理中心

-----正在爬取第1页，第27731条微博------
-----正在爬取第1页，第27732条微博------
-----正在爬取第1页，第27733条微博------
-----正在爬取第1页，第27734条微博------
-----正在爬取第1页，第27735条微博------
-----正在爬取第1页，第27736条微博------
-----正在爬取第1页，第27737条微博------
-----正在爬取第1页，第27738条微博------
-----正在爬取第1页，第27739条微博------
-----正在爬取第1页，第27740条微博------
-----正在爬取第2页，第27741条微博------
-----正在爬取第2页，第27742条微博------
-----正在爬取第2页，第27743条微博------
-----正在爬取第2页，第27744条微博------
-----正在爬取第2页，第27745条微博------
-----正在爬取第2页，第27746条微博------
-----正在爬取第2页，第27747条微博------
----

-----正在爬取第10页，第27991条微博------
-----正在爬取第10页，第27992条微博------
-----正在爬取第10页，第27993条微博------
-----正在爬取第10页，第27994条微博------
-----正在爬取第10页，第27995条微博------
-----正在爬取第10页，第27996条微博------
-----正在爬取第10页，第27997条微博------
-----正在爬取第10页，第27998条微博------
-----正在爬取第10页，第27999条微博------
-----正在爬取第10页，第28000条微博------
-----正在爬取第11页，第28001条微博------
-----正在爬取第11页，第28002条微博------
-----正在爬取第11页，第28003条微博------
-----正在爬取第11页，第28004条微博------
-----正在爬取第11页，第28005条微博------
-----正在爬取第11页，第28006条微博------
-----正在爬取第11页，第28007条微博------
-----正在爬取第11页，第28008条微博------
-----正在爬取第11页，第28009条微博------
-----正在爬取第11页，第28010条微博------
-----正在爬取第12页，第28011条微博------
-----正在爬取第12页，第28012条微博------
-----正在爬取第12页，第28013条微博------
-----正在爬取第12页，第28014条微博------
-----正在爬取第12页，第28015条微博------
-----正在爬取第12页，第28016条微博------
-----正在爬取第12页，第28017条微博------
-----正在爬取第12页，第28018条微博------
-----正在爬取第12页，第28019条微博------
-----正在爬取第13页，第28020条微博------
-----正在爬取第13页，第28021条微博------
-----正在爬取第13页，第28022条微博------
-----正在爬取第13页，第28023条微博------
-----正在爬取第

-----正在爬取第39页，第28265条微博------
-----正在爬取第39页，第28266条微博------
-----正在爬取第39页，第28267条微博------
-----正在爬取第39页，第28268条微博------
-----正在爬取第39页，第28269条微博------
-----正在爬取第39页，第28270条微博------
-----正在爬取第39页，第28271条微博------
-----正在爬取第39页，第28272条微博------
-----正在爬取第39页，第28273条微博------
-----正在爬取第40页，第28274条微博------
-----正在爬取第40页，第28275条微博------
-----正在爬取第40页，第28276条微博------
-----正在爬取第40页，第28277条微博------
-----正在爬取第40页，第28278条微博------
-----正在爬取第40页，第28279条微博------
-----正在爬取第40页，第28280条微博------
-----正在爬取第40页，第28281条微博------
-----正在爬取第40页，第28282条微博------
-----正在爬取第41页，第28283条微博------
-----正在爬取第41页，第28284条微博------
-----正在爬取第41页，第28285条微博------
-----正在爬取第41页，第28286条微博------
-----正在爬取第41页，第28287条微博------
-----正在爬取第41页，第28288条微博------
-----正在爬取第41页，第28289条微博------
-----正在爬取第41页，第28290条微博------
-----正在爬取第41页，第28291条微博------
-----正在爬取第41页，第28292条微博------
-----正在爬取第42页，第28293条微博------
-----正在爬取第42页，第28294条微博------
-----正在爬取第42页，第28295条微博------
-----正在爬取第42页，第28296条微博------
-----正在爬取第42页，第28297条微博------
-----正在爬取第

-----正在爬取第68页，第28539条微博------
-----正在爬取第68页，第28540条微博------
-----正在爬取第68页，第28541条微博------
-----正在爬取第68页，第28542条微博------
-----正在爬取第68页，第28543条微博------
-----正在爬取第68页，第28544条微博------
-----正在爬取第68页，第28545条微博------
-----正在爬取第68页，第28546条微博------
-----正在爬取第68页，第28547条微博------
-----正在爬取第69页，第28548条微博------
-----正在爬取第69页，第28549条微博------
-----正在爬取第69页，第28550条微博------
-----正在爬取第69页，第28551条微博------
-----正在爬取第69页，第28552条微博------
-----正在爬取第69页，第28553条微博------
-----正在爬取第69页，第28554条微博------
-----正在爬取第69页，第28555条微博------
-----正在爬取第69页，第28556条微博------
-----正在爬取第70页，第28557条微博------
-----正在爬取第70页，第28558条微博------
-----正在爬取第70页，第28559条微博------
-----正在爬取第70页，第28560条微博------
-----正在爬取第70页，第28561条微博------
-----正在爬取第70页，第28562条微博------
-----正在爬取第70页，第28563条微博------
-----正在爬取第70页，第28564条微博------
-----正在爬取第70页，第28565条微博------
-----正在爬取第70页，第28566条微博------
-----正在爬取第71页，第28567条微博------
-----正在爬取第71页，第28568条微博------
-----正在爬取第71页，第28569条微博------
-----正在爬取第71页，第28570条微博------
-----正在爬取第71页，第28571条微博------
-----正在爬取第